In [42]:
from sklearn.preprocessing import OneHotEncoder
from pymongo import MongoClient
from zipfile import ZipFile, ZIP_DEFLATED
from sets import Set
import numpy as np
import json, sys, os, time

In [2]:
local = False
if local:
    client = MongoClient('localhost', 27017)
    plan_col = client.aca.plan
    drug_col = client.aca.drug
else:
    client = MongoClient('ec2-54-153-83-172.us-west-1.compute.amazonaws.com', 27017)
    plan_col = client.plans.plans
    drug_col = client.formularies.drugs
    prov_col = client.providers.providers
    faci_col = client.providers.facilities

# plan_col.insert_many(json.load(open('ak_plan.json')))
# plan_col.insert_many(json.load(open('or_plan.json')))
# drug_col.insert_many(json.load(open('ak_drug.json')))
# drug_col.insert_many(json.load(open('or_drug.json')))
print 'Using %s Mongo, total drug: %d, total plan: %d' %('local' if local else 'aws', 
                                                         drug_col.count(), plan_col.count())
# client.formularies.scollection_names()
# client.providers.collection_names()

Using aws Mongo, total drug: 1540473, total plan: 12242


In [57]:
print len(drug_col.distinct('plans.plan_id'))

6035


In [26]:
# i = 0
# for p in plan_col.aggregate([{"$group": {"_id": "$plan_id", "count": {"$sum": 1}}}]):
#     if p['count'] > 5:
# #         i += 1
#         print p['_id'],p['count']
# #         break
# i

i = 0
for p in drug_col.aggregate([{"$group": {"_id": "$rxnorm_id", "count": {"$sum": 1}}}]):
    if p['count'] > 20:
#         i += 1
        print p['_id'],p['count']
#         break
i

727954 91
1087399 29
1314672 52
197575 353
897586 65
860918 392
403923 227
828353 54
352045 218
197606 498
729931 86
197604 440
897676 89
1598637 101
237372 98
198078 303
904601 451
312085 297
1660929 108
1657196 32
977923 186
762005 216
1193345 204
1596780 171
209339 83
672920 190
800400 43
903316 202
861353 392
208137 48
861509 42
206620 61
1298356 64
314285 515
861156 370
977842 207
1189678 22
1552004 127
205860 91
197304 326
208551 138
404444 219
755357 100
1376084 30
1314768 100
1716037 122
1250486 25
1099563 638
858071 158
856467 114
201848 66
848180 349
238004 227
836378 34
1738808 39
1232714 194
226719 55
867381 54
1720612 93
313572 65
1359271 202
855906 613
402244 32
201905 67
1486198 105
897590 243
1591967 57
197944 541
1191060 40
999653 98
847630 23
198193 724
1246113 54
314231 1153
213271 62
1488675 124
607024 209
731229 196
1365660 91
1191394 33
1594432 82
282609 64
836358 554
861356 292
884522 151
213442 211
997230 70
310174 94
992668 210
855926 751
865123 329
197931 319


0

In [31]:
# for p in plan_col.find({'plan_id':'22444PA0010006'}):
#     print p,'\n'
i = 0
for d in drug_col.find({'rxnorm_id':'855906'}):
    i+=1
    print d,'\n'
    if i==25:
        break

{u'drug_name': u'DICLOFENAC   TAB 50MG DR', u'_id': ObjectId('5778743a2ccfd95627c2dd58'), u'rxnorm_id': u'855906', u'plans': [{u'quantity_limit': False, u'plan_id': u'19304NH0010001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'GENERIC'}, {u'quantity_limit': False, u'plan_id': u'19304NH0010002', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'GENERIC'}, {u'quantity_limit': False, u'plan_id': u'19304NH0010003', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'GENERIC'}, {u'quantity_limit': False, u'plan_id': u'19304NH0010004', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'GENERIC'}, {u'quantity_limit': False, u'plan_id': u'19304NH0010005', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'GENERIC'}, {u'q

###Load encode definition

In [4]:
def getEncodeFields(encode_def, rtn, path=''):
    ''' extract the selected fields from the encode json definition '''
    
    if 'encode' in encode_def and encode_def['encode'] == 1:
        rtn[path[1:]] = encode_def['type']
    elif 'encode' not in encode_def:
        for f in encode_def:
            getEncodeFields(encode_def[f], rtn, path + ('' if f=='properties' else '.'+f))
    return rtn

encode_list = getEncodeFields(json.load(open('encode2.json')), {})
encode_list

{u'drug.plans.drug_tier': u'string',
 u'drug.plans.prior_authorization': u'boolean',
 u'drug.plans.quantity_limit': u'boolean',
 u'drug.plans.step_therapy': u'boolean',
 u'plan.formulary.cost_sharing.coinsurance_opt': u'string',
 u'plan.formulary.cost_sharing.coinsurance_rate': u'float',
 u'plan.formulary.cost_sharing.copay_amount': u'float',
 u'plan.formulary.cost_sharing.copay_opt': u'string',
 u'plan.formulary.cost_sharing.pharmacy_type': u'string',
 u'plan.formulary.drug_tier': u'string',
 u'plan.formulary.mail_order': u'boolean',
 u'plan.network.network_tier': u'string',
 u'plan.plan_id_type': u'string'}

###Get feature space

In [5]:
# feature_space = {k:client.aca[k.split('.')[0]].distinct(k[k.index('.')+1:]) 
#                  for k,v in encode_list.items() if v=='string'}

feature_space = {k:(plan_col if k.startswith('plan') else drug_col).distinct(k[k.index('.')+1:]) 
                 for k,v in encode_list.items() if v=='string'}

feature_space

{u'drug.plans.drug_tier': [u'ZERO-COST-SHARE-PREVENTIVE',
  u'GENERIC',
  u'PREFERRED',
  u'GENERIC-BRAND-SPECIALITY',
  u'GENERIC-BRAND-SPECIALITY-DRUGS',
  u'GENERIC-BRAND',
  u'NON-PREFERRED-BRAND',
  u'NON-PREFERRED-GENERIC',
  u'SPECIALITY-DRUGS',
  u'PREFERRED-BRAND',
  u'BRAND',
  u'PREFERRED-GENERIC',
  u'NON-PREFERRED-GENERIC-BRAND',
  u'ZERO-COST-PREVENTIVE-DRUGS',
  u'SPECIALTY-DRUGS',
  u'MEDICAL-SERVICE-DRUGS',
  u'SELECT',
  u'SPECIALTY',
  u'PREVENTIVE',
  u'VALUE',
  u'MEDICAL-SERVICE',
  u'NON-PREFERRED-GENERIC-NON-PREFERRED-BRAND',
  u'ZERO-COST-SHARE-PREVENTIVE-DRUGS',
  u'NON-PREFERRED',
  u'PREFERRED-BRAND-SPECIALTY-DRUGS',
  u'GENERIC-SPECIALTY-DRUGS',
  u'NON-PREFERRED-BRAND-SPECIATLY-DRUGS',
  u'TIER-ONE',
  u'TIER-THREE',
  u'TIER-TWO',
  u'TIER-FOUR',
  u'FORMULARY-DRUGS',
  u'NONPREFERRED-BRAND',
  u'PREVENTIVE-ACA',
  u'PREFERRED-GENERIC-PREFERRED-BRAND',
  u'NON-PREFERRED-BRAND-SPECIALTY-DRUGS',
  u'GENERIC-BRAND-SPECIALTY',
  u'NON-PREFERRED-SPECIALTY',
  

In [12]:
# for p in plan_col.find().limit(5):
#     print p
# drug_col.distinct('drug.plans.prior_authorization')
# len(drug_col.distinct('rxnorm_id'))
# drug_col.count()
# all_plans
if True:
    sys.exit('stop')
print 'test print'

# common_drug

SystemExit: stop

To exit: use 'exit', 'quit', or Ctrl-D.


###Get common drugs between plans

In [34]:
# get rxnorm_id group for each plan
all_plans = drug_col.distinct('plans.plan_id')
plan_drug = [drug_col.find({'plans.plan_id':pid}) for pid in all_plans] # if 'MI' in pid] 
n_plan = len(all_plans)

# find the first plan with non-zero drug association, 
# otherwise 'i' is out-of-bound of plan_drug and will stop 'naturally'
print 'total plan: %d' %n_plan
cnt, i = [], -1
while(len(cnt) == 0):
    i += 1    
    cnt = plan_drug[i].distinct('rxnorm_id')   

common_drug, n_empty = Set(cnt), i

print 'checking common drug among for %d plans ...' %(n_plan-i)
for pd in plan_drug[i+1:]:
    i += 1
    rx = pd.distinct('rxnorm_id')
    if len(rx)>0:
        common_drug.intersection_update(rx)
    else:
        n_empty += 1
    if i%300 == 0:
        print 'finishing %d plans, %d without drug, %d common drugs ...' %(i, n_empty, len(common_drug))

print 'plan without drug: %d' %n_empty

# common_drug and drug_attr list will ensure the order of drug/attribute combination remains unchange
common_drug = list(common_drug)
n_drug = len(common_drug)
drug_attr = [[k.split('.')[-1],v,k] for k,v in encode_list.items() if k.startswith ('drug')]
drug_cat_index = [k[1]=='string' for k in drug_attr]*n_drug

print 'There are %d common drug between %d plans!' %(n_drug, n_plan-n_empty)

In [56]:
t0 = time.time()
print 'drugs: %d' %len(drug_col.find({'plans.plan_id':'67577MI0390012'}).distinct('rxnorm_id'))
print("--- %s seconds ---" % (time.time() - t0))

drugs: 2710
--- 19.570182085 seconds ---


In [53]:
len([i for i in all_plans if 'MI' in i])

236

###Get pharmacy_type space (over all plans) for each drug_tier

In [6]:
# drug tiers: SELECT, u'PREFERRED', u'BRAND', u'SPECIALTY', u'PREVENTIVE', u'VALUE', u'MEDICAL-SERVICE-DRUGS
# NOTE: seems there is a bug for the distinct query with sub-array $elemMatch query, below doesn't work
# tier_pharm = {tier : plan_col.find(
#         {'formulary.drug_tier':tier},
#         {'_id':0, 'formulary':{'$elemMatch':{'drug_tier':tier}}, 'formulary.cost_sharing.pharmacy_type':1}
#     ).distinct('formulary.cost_sharing.pharmacy_type')
#     for tier in feature_space['drug.plans.drug_tier']}

# doing this in traditional way
tier_pharm = {}
for tier in feature_space['drug.plans.drug_tier']:
    query = plan_col.find(
        {'formulary.drug_tier':tier}, 
        {'_id':0, 'formulary':{'$elemMatch':{'drug_tier':tier}}, 'formulary.cost_sharing.pharmacy_type':1})
    tier_pharm[tier] = list(Set(cs['pharmacy_type'] for p in query for cs in p['formulary'][0]['cost_sharing']))

tier_pharm


{u'BRAND': [u'1-MONTH-OUT-RETAIL',
  u'3-MONTH-IN-MAIL',
  u'3-MONTH-IN-RETAIL',
  u'1-MONTH-IN-RETAIL',
  u'3-MONTH-OUT-RETAIL'],
 u'MEDICAL-SERVICE-DRUGS': [u'1-MONTH-IN-RETAIL', u'3-MONTH-IN-RETAIL'],
 u'PREFERRED': [u'1-MONTH-OUT-RETAIL',
  u'3-MONTH-IN-MAIL',
  u'3-MONTH-IN-RETAIL',
  u'1-MONTH-IN-RETAIL',
  u'3-MONTH-OUT-RETAIL'],
 u'PREVENTIVE': [u'1-MONTH-IN-RETAIL', u'3-MONTH-IN-RETAIL'],
 u'SELECT': [u'1-MONTH-OUT-RETAIL',
  u'3-MONTH-IN-MAIL',
  u'3-MONTH-IN-RETAIL',
  u'1-MONTH-IN-RETAIL',
  u'3-MONTH-OUT-RETAIL'],
 u'SPECIALTY': [u'1-MONTH-IN-RETAIL'],
 u'VALUE': [u'1-MONTH-OUT-RETAIL',
  u'3-MONTH-IN-MAIL',
  u'3-MONTH-IN-RETAIL',
  u'1-MONTH-IN-RETAIL',
  u'3-MONTH-OUT-RETAIL']}

###Get unique pharmacy_type for each drug_tier

In [7]:
# put tier names into list so the order is fixed for feature extraction
tiers = tier_pharm.keys()
# we build pharmacy type into the order of feature vector, so no need to include
cost_attr = [[k.split('.')[-1],v,k] for k,v in encode_list.items() if 'cost_sharing' in k and 'pharmacy_type' not in k]
# flatten the vector to combine all tiers
cost_cat_index = [y for x in [[k[1]=='string' for k in cost_attr]*len(tier_pharm[t]) for t in tiers] for y in x]
# plan level attributes
plan_attr = [[k.split('.')[-1],v,k] for k,v in encode_list.items() if k.startswith('plan') and 'formulary' not in k]
plan_cat_index = [a[1]=='string' for a in plan_attr]
# total feature catagrical index - must match with the order of feature canconnation in plan
cat_index = cost_cat_index + drug_cat_index

###Extraction integer features for each plan
- get plan feature --> cost_sharing/drug_tier
- get drug feature --> pharmacy type

In [9]:
# for each plan, get int features (plan level & combined fomulary level)
plan_int_feature = {}
for pid in all_plans:
    # extract plan feature
    plan_data = plan_col.find_one({'plan_id':pid})
    
    
    # TODO: check if network tier is necessary
    # plan_data['network'][0]['network_tier']
    
    #drop the plan if it doesn't have complete drug tiers set (7)
    if len(plan_data['formulary']) != len(tier_pharm):
        print 'plan %s doesn\'t have complete Drug Tier info: %d tiers' %(pid, len(plan_data['formulary']))
        continue

    # for the formulary, build feature for each drug_tier
    tier_feature = [None]*len(tiers)
    for tier in plan_data['formulary']:
        # for each drug_tier, expland cost_sharing for all pharmacy_type
        cost_feature = [None]*len(tier_pharm[tier['drug_tier']])
        for cs in tier['cost_sharing']:            
            ph_idx = tier_pharm[tier['drug_tier']].index(cs['pharmacy_type'])
            cost_feature[ph_idx] = [cs[a[0]] if a[1]!='string' 
                                else feature_space[a[2]].index(cs[a[0]]) 
                                for a in cost_attr]
        tier_feature[tiers.index(tier['drug_tier'])] = [c if c else [None]*len(cost_attr) for c in cost_feature]        
    # flaten the vector for the plan from hierarchy: tier-cost-pharmacy
    formulary_feature = [z for x in tier_feature for y in x for z in y]
    
    
    # get the list of drug attributes for a plan
    drug_cur = drug_col.find(
        {'plans.plan_id':pid, 'rxnorm_id':{'$in':common_drug}}, 
        {'_id':0, 'rxnorm_id':1, 'plans':{ '$elemMatch':{'plan_id':pid} }}
    )

    drug_dict = {d['rxnorm_id']:d['plans'][0] for d in drug_cur} #TODO: choose the plan matches the tier

    # drop the plan if it doesn't match with common drug list
    if len(drug_dict) != n_drug:
        print 'plan %s drug list is nonnormal (n=%d), skip' %(pid, len(drug_dict))
        continue

    # flat the drug attributes for all common drugs    
    drug_feature = [drug_dict[rx][attr[0]] if attr[1]!='string' 
                             else feature_space[attr[2]].index(drug_dict[rx][attr[0]])
                             for rx in common_drug for attr in drug_attr]
    
    # combine for plan feature - must match with catagroical index concannation order
    plan_int_feature[pid] = formulary_feature + drug_feature


plan 21989AK0030001 doesn't have complete Drug Tier info: 0 tiers
plan 21989AK0050001 doesn't have complete Drug Tier info: 0 tiers
plan 21989AK0050002 doesn't have complete Drug Tier info: 0 tiers
plan 21989AK0070001 doesn't have complete Drug Tier info: 0 tiers
plan 28415OR0210001 doesn't have complete Drug Tier info: 0 tiers
plan 28415OR0010001 doesn't have complete Drug Tier info: 0 tiers
plan 28415OR0010002 doesn't have complete Drug Tier info: 0 tiers
plan 28415OR0050001 doesn't have complete Drug Tier info: 0 tiers


In [4]:
# [len(x) for x in plan_int_feature.values()]
# len(cost_cat_index + drug_cat_index)
# len(plan_col.distinct('plan_id'))
# plan_url = Set()
# with open('results-plan-urls.txt') as urls:
#     for url in urls.readlines():
#         plan_url.add(json.loads(url)['url'])


drug_url = Set()
with open('results-formulary-urls.txt') as urls:
    for url in urls.readlines():
        drug_url.add(json.loads(url)['url'])

len(drug_url)

372

In [9]:
drug_url = list(drug_url)
drug_url[100]

u'https://provider-search.qualchoice.com/drugs.json'

In [90]:
cmd = []
fi = 1
for u in plan_url:
    cmd.append('curl -o Plan_Json/f%d.json %s' %(fi, u))
    fi += 1

In [92]:
# import os
for c in cmd:
    print 'executing %s' %c
    os.system(c)

executing curl -o Plan_Json/f1.json https://www.ibx.com/scripts/custom/cms-data/ibc-khpe/plans-33871.json
executing curl -o Plan_Json/f2.json https://www.caresource.com/vendor/cms/data/20160615/Plans_CareSource WV_09302015_1256.json
executing curl -o Plan_Json/f3.json https://download.getjsonfile.com/data/plans/plans_12028.json
executing curl -o Plan_Json/f4.json http://www.mdwise.org/MediaLibraries/MDwise/Files/json/plans.json
executing curl -o Plan_Json/f5.json https://www.zoomcare.com/cms/plans.json
executing curl -o Plan_Json/f6.json https://www.bcbsga.com/health-insurance/documents/getmachinereadable?FILENAME=PLANS_GA&STATE=GA
executing curl -o Plan_Json/f7.json https://www.healthplex.com/FFM-DATA/NJ/plans.json
executing curl -o Plan_Json/f8.json https://download.getjsonfile.com/data/plans/plans_67190.json
executing curl -o Plan_Json/f9.json https://wsprod.deltadental.com/CMSDirectory/plans.json
executing curl -o Plan_Json/f10.json https://www.goldendentalplans.com/plans/select/js

In [109]:
# os.system(cmd[70])
for z in [11,25,27,47,67,114,173,49,160]:
    print cmd[z]


curl -o Plan_Json/f12.json http://fm.formularynavigator.com/jsonFiles/publish/150/69/plans.json
curl -o Plan_Json/f26.json https://www.dentaquest.com/getattachment/Marketplace/CMSDataIndex/DQ_Plans.json
curl -o Plan_Json/f28.json http://fm.formularynavigator.com/jsonFiles/publish/7/85/plans.json
curl -o Plan_Json/f48.json http://fm.formularynavigator.com/jsonFiles/publish/11/47/plans.json
curl -o Plan_Json/f68.json http://fm.formularynavigator.com/jsonFiles/publish/11/45/plans.json
curl -o Plan_Json/f115.json https://www.dentaquest.com/getattachment/Marketplace/DentalCarePlus/DCP_Plans.json
curl -o Plan_Json/f174.json http://fm.formularynavigator.com/jsonFiles/publish/137/80/plans.json
curl -o Plan_Json/f50.json https://secure.ccok.com/json/plans.json
curl -o Plan_Json/f161.json https://s3.amazonaws.com/minuteman-machine-readable/MH_Plan.json


In [112]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('Plan_Json/') if isfile(join('Plan_Json/', f))]

In [122]:
plan_col = MongoClient('localhost',27017).aca.plan
plan_col.delete_many({})
for f in onlyfiles:
    if f.startswith('f'):
        try:
            plan_col.insert_many(json.load(open('Plan_Json/'+f)))
        except:
            print 'failed to add '+f
    

failed to add f107.json
failed to add f132.json
failed to add f161.json
failed to add f171.json
failed to add f19.json
failed to add f2.json
failed to add f34.json
failed to add f41.json
failed to add f50.json
failed to add f52.json
failed to add f62.json
failed to add f73.json
failed to add f76.json
failed to add f93.json
failed to add f97.json


In [134]:
for p in plan_col.find({'plan_id':'33871PA0100002'}):
    print p['formulary'],'\n'

[{u'drug_tier': u'GENERIC', u'mail_order': True, u'cost_sharing': [{u'copay_amount': 7.0, u'pharmacy_type': u'1-MONTH-IN-RETAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}, {u'copay_amount': 7.0, u'pharmacy_type': u'1-MONTH-IN-MAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}, {u'copay_amount': 14.0, u'pharmacy_type': u'3-MONTH-IN-MAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}, {u'copay_amount': 0.0, u'pharmacy_type': u'1-MONTH-OUT-RETAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.7, u'coinsurance_opt': u'NO-CHARGE'}]}, {u'drug_tier': u'NON-PREFERRED-BRAND', u'mail_order': True, u'cost_sharing': [{u'copay_amount': 75.0, u'pharmacy_type': u'1-MONTH-IN-RETAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}, {u'copay_amount': 75.0, u'pharmacy_type': u'1-MONTH-IN-RETAIL', u'copay_opt': u'NO-CHARGE', u'

In [238]:
client.close()